In [1]:
import pandas as pd

# Read the file
df = pd.read_csv('feature_extract.csv')

# Basic info
print(df.shape)
print(df.columns)


(10080, 17)
Index(['Event', 'Site', 'Result', 'WhiteElo', 'BlackElo', 'ECO', 'Opening',
       'NumMoves', 'Moves', 'Termination', 'game_format', 'is_tournament',
       'UTCDatetime', 'username', 'Colour', 'Eval_List', 'Material_Diff_List'],
      dtype='object')


In [2]:
import pandas as pd
import numpy as np

# Step 1 — Read the file
df = pd.read_csv('feature_extract.csv')

# Safety check
assert 'username' in df.columns, "Expected 'username' column not found in dataset."

# Step 2 — Compute ELO difference (opponent - player)
def compute_elo_diff(row):
    if row['Colour'].lower() == 'white':
        return row['BlackElo'] - row['WhiteElo']
    else:
        return row['WhiteElo'] - row['BlackElo']

df['EloDiff'] = df.apply(compute_elo_diff, axis=1)

# Step 3 — Convert result to numeric
def get_result(row):
    if row['Result'] == '1-0' and row['Colour'].lower() == 'white':
        return 1
    elif row['Result'] == '0-1' and row['Colour'].lower() == 'black':
        return 1
    elif row['Result'] == '1/2-1/2':
        return 0.5
    else:
        return 0

df['Score'] = df.apply(get_result, axis=1)

# Step 4 — Aggregate per player
grouped = df.groupby('username')

summary = grouped.agg(
    MeanMoves=('NumMoves', 'mean'),
    StdMoves=('NumMoves', 'std'),
    MeanEloDiff=('EloDiff', 'mean')
).reset_index()

# Step 5 — Compute win-rate features
player_stats = []
for name, group in grouped:
    total_games = len(group)
    total_score = group['Score'].sum()

    win_rate = (total_score / total_games) * 100

    pos_group = group[group['EloDiff'] > 0]
    win_rate_pos = (pos_group['Score'].sum() / len(pos_group) * 100) if len(pos_group) > 0 else np.nan

    neg_group = group[group['EloDiff'] < 0]
    win_rate_neg = (neg_group['Score'].sum() / len(neg_group) * 100) if len(neg_group) > 0 else np.nan

    player_stats.append({
        'username': name,
        'WinRate': win_rate,
        'WinRate_PosEloDiff': win_rate_pos,
        'WinRate_NegEloDiff': win_rate_neg
    })

player_stats_df = pd.DataFrame(player_stats)

# --- Make sure merge key exists ---
assert 'username' in summary.columns and 'username' in player_stats_df.columns, "Merge key missing in one of the dataframes."

# Step 6 — Merge and export
training_data = pd.merge(summary, player_stats_df, on='username')
training_data.to_csv('training_data.csv', index=False)

print("✅ training_data.csv created successfully!")
print(training_data.head())


✅ training_data.csv created successfully!
   username  MeanMoves   StdMoves  MeanEloDiff  WinRate  WinRate_PosEloDiff  \
0   2700172    37.2500  15.813496      15.6250   31.250           40.000000   
1       AB2    32.3750  13.657110     -40.9375   50.000           20.000000   
2      ABUN    51.4375  16.997917     -86.0625   65.625           50.000000   
3  ADRIRIEN    25.9375  10.592253      44.8125   50.000           50.000000   
4      AJTO    31.8125  11.136838      44.8125   15.625           11.538462   

   WinRate_NegEloDiff  
0           16.666667  
1           63.636364  
2           72.727273  
3           50.000000  
4           33.333333  


In [7]:
import pandas as pd
import chess
from stockfish import Stockfish
import json

# --- CONFIGURATION ---
STOCKFISH_PATH = r"D:\IIT Roorkee\Sem 5\AI_ML\Project\Stockfish\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"
START_INDEX = 7600   # game index to start from (0-based)
BATCH_SIZE = 1200    # number of new games to process

# Initialize Stockfish
stockfish = Stockfish(path=STOCKFISH_PATH, parameters={"Threads": 2, "Skill Level": 20})

# --- Helper functions ---
def get_material_diff(board: chess.Board) -> int:
    piece_values = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9}
    material = 0
    for piece_type, val in piece_values.items():
        material += len(board.pieces(piece_type, chess.WHITE)) * val
        material -= len(board.pieces(piece_type, chess.BLACK)) * val
    return material

def safe_eval(fen: str) -> int:
    stockfish.set_fen_position(fen)
    eval_data = stockfish.get_evaluation()
    if eval_data["type"] == "cp":
        value = eval_data["value"]
    else:
        value = 1000 if eval_data["value"] > 0 else -1000
    return max(min(value, 1000), -1000)

# --- Main processing ---
data = pd.read_csv("feature_extract.csv")

for i in range(START_INDEX, min(START_INDEX + BATCH_SIZE, len(data))):
    # Skip already processed rows (if they already have values)
    if not pd.isna(data.loc[i, "Eval_List"]) and not pd.isna(data.loc[i, "Material_Diff_List"]):
        continue

    moves_str = data.loc[i, "Moves"]
    moves = moves_str.split()

    board = chess.Board()
    eval_list = []
    material_diff_list = []

    for move in moves:
        eval_cp = safe_eval(board.fen())
        material_diff = get_material_diff(board)

        eval_list.append(eval_cp)
        material_diff_list.append(material_diff)

        try:
            board.push_san(move)
        except Exception:
            break

    data.loc[i, "Eval_List"] = json.dumps(eval_list)
    data.loc[i, "Material_Diff_List"] = json.dumps(material_diff_list)
    print(f"✅ Processed game {i+1}/{len(data)}")

# Save progress
data.to_csv("feature_extract.csv", index=False)
print(f"✅ Saved evaluations for games {START_INDEX+1}–{min(START_INDEX+BATCH_SIZE, len(data))}")


✅ Processed game 7601/10080
✅ Processed game 7602/10080
✅ Processed game 7603/10080
✅ Processed game 7604/10080
✅ Processed game 7605/10080
✅ Processed game 7606/10080
✅ Processed game 7607/10080
✅ Processed game 7608/10080
✅ Processed game 7609/10080
✅ Processed game 7610/10080
✅ Processed game 7611/10080
✅ Processed game 7612/10080
✅ Processed game 7613/10080
✅ Processed game 7614/10080
✅ Processed game 7615/10080
✅ Processed game 7616/10080
✅ Processed game 7617/10080
✅ Processed game 7618/10080
✅ Processed game 7619/10080
✅ Processed game 7620/10080
✅ Processed game 7621/10080
✅ Processed game 7622/10080
✅ Processed game 7623/10080
✅ Processed game 7624/10080
✅ Processed game 7625/10080
✅ Processed game 7626/10080
✅ Processed game 7627/10080
✅ Processed game 7628/10080
✅ Processed game 7629/10080
✅ Processed game 7630/10080
✅ Processed game 7631/10080
✅ Processed game 7632/10080
✅ Processed game 7633/10080
✅ Processed game 7634/10080
✅ Processed game 7635/10080
✅ Processed game 763

In [11]:
print(data.head(1)[['Moves','Eval_List','Material_Diff_List']])

                                               Moves  \
0  d4 d5 c4 Nf6 e3 c6 Nc3 e6 Bd3 Bb4 Bd2 Nbd7 e4 ...   

                                           Eval_List  \
0  [32, 24, 24, 30, 66, 5, 32, 29, 35, 24, 43, 31...   

                                  Material_Diff_List  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


In [12]:
import pandas as pd
import json
import numpy as np

def compute_average_centipawn_loss(eval_list, color="white"):
    """
    Compute average centipawn loss (ACPL) for the given color.
    eval_list contains evaluations BEFORE each move.
    """
    evals = np.array(eval_list, dtype=float)
    if len(evals) < 2:
        return np.nan

    losses = []
    for i in range(len(evals) - 1):
        move_color = "white" if i % 2 == 0 else "black"
        delta = evals[i + 1] - evals[i]

        if move_color == color:
            # Convert delta to that player's perspective:
            if color == "white":
                # White's eval got worse if delta < 0
                loss = max(0, -delta)
            else:
                # For Black, eval increases (from White POV) means loss
                loss = max(0, delta)
            losses.append(loss)

    return np.mean(losses) if losses else np.nan


# --- Example usage ---
data = pd.read_csv("feature_extract.csv")
eval_list = json.loads(data.loc[2, "Eval_List"])

acpl_white = compute_average_centipawn_loss(eval_list, "white")
acpl_black = compute_average_centipawn_loss(eval_list, "black")

print(f"Average Centipawn Loss (White): {acpl_white:.2f}")
print(f"Average Centipawn Loss (Black): {acpl_black:.2f}")



Average Centipawn Loss (White): 94.07
Average Centipawn Loss (Black): 80.37


In [15]:
import pandas as pd
import json
import numpy as np

# --- Helper: ACPL computation ---
def compute_average_centipawn_loss(eval_list, color="white"):
    evals = np.array(eval_list, dtype=float)
    if len(evals) < 2:
        return np.nan

    losses = []
    for i in range(len(evals) - 1):
        move_color = "white" if i % 2 == 0 else "black"
        delta = evals[i + 1] - evals[i]

        if move_color == color:
            if color == "white":
                loss = max(0, -delta)
            else:
                loss = max(0, delta)
            losses.append(loss)

    return np.mean(losses) if losses else np.nan


# --- Load data ---
extract_df = pd.read_csv("feature_extract.csv")
training_df = pd.read_csv("training_data.csv")

# --- Config ---
COLOR = "white"   # or "black"
GAMES_PER_PLAYER = 15
MAX_PLAYERS = 275  # ✅ only first 275 players

# --- Process each player ---
players = extract_df["username"].unique()[:MAX_PLAYERS]  # ✅ restrict to first 275

for player in players:
    player_games = extract_df[extract_df["username"] == player].head(GAMES_PER_PLAYER)
    acpl_list = []
    variance_per_game = []
    after_opening_acpl_list = []

    for _, row in player_games.iterrows():
        try:
            eval_list = json.loads(row["Eval_List"])
        except (TypeError, json.JSONDecodeError):
            continue

        if len(eval_list) < 2:
            continue

        # Compute full-game ACPL
        acpl = compute_average_centipawn_loss(eval_list, COLOR)
        acpl_list.append(acpl)

        # Compute variance per game
        acpl_diffs = []
        for i in range(len(eval_list) - 1):
            move_color = "white" if i % 2 == 0 else "black"
            delta = eval_list[i + 1] - eval_list[i]
            if move_color == COLOR:
                loss = max(0, -delta) if COLOR == "white" else max(0, delta)
                acpl_diffs.append(loss)
        if len(acpl_diffs) > 1:
            variance_per_game.append(np.var(acpl_diffs))

        # Compute After-opening ACPL (moves after 20)
        if len(eval_list) > 40:  # 20 moves = 40 half-moves
            after_opening_part = eval_list[40:]
            after_opening_acpl = compute_average_centipawn_loss(after_opening_part, COLOR)
            after_opening_acpl_list.append(after_opening_acpl)

    # Aggregate metrics
    mean_acpl = np.nanmean(acpl_list)
    std_acpl = np.nanstd(acpl_list)
    mean_game_variance = np.nanmean(variance_per_game)
    after_opening_acpl = np.nanmean(after_opening_acpl_list) if after_opening_acpl_list else np.nan

    # Update training data
    training_df.loc[
        training_df["username"] == player, 
        ["Mean_ACPL", "Std_ACPL", "Mean_Game_ACPL_Variance", "After_opening_ACPL"]
    ] = [mean_acpl, std_acpl, mean_game_variance, after_opening_acpl]

    print(f"✅ {player} processed.")

# --- Save updated training data ---
training_df.to_csv("training_data.csv", index=False)
print("✅ ACPL features updated successfully for first 275 players!")


✅ Onetreehill processed.
✅ acaya processed.
✅ bastianmunz processed.
✅ Komarudin-Garut processed.
✅ Gregor_Baron processed.
✅ aerdna87 processed.
✅ Lu4ik processed.
✅ phet processed.
✅ inthebighooy processed.
✅ AceWestfall processed.
✅ izzyreal processed.
✅ comaturbo processed.
✅ Crytash processed.
✅ mariomaster processed.
✅ jebisesome processed.
✅ mhmdkarimi13 processed.
✅ Ivorybags processed.
✅ GeorgyM89 processed.
✅ TJK125HQF processed.
✅ yanaaa processed.
✅ girat processed.
✅ cipo1951 processed.
✅ rolling1 processed.
✅ sting47 processed.
✅ SiavashHuman processed.
✅ Dr_Perdyllo processed.
✅ Misha19701309 processed.
✅ BoskoRadic77 processed.
✅ ThreeFoxes processed.
✅ ch1ll processed.
✅ Tramtararam processed.
✅ bhram1345 processed.
✅ TEMPOchess processed.
✅ Hard2kill74 processed.
✅ geek46 processed.
✅ Snowden processed.
✅ Aurimas123 processed.
✅ celiktas69 processed.
✅ arlequeen processed.
✅ jmaltby processed.
✅ genoreling processed.
✅ ElysiumKing processed.
✅ hritic333 processed.
✅ M

In [18]:
import pandas as pd

# --- Load datasets ---
extract_df = pd.read_csv("feature_extract.csv")
training_df = pd.read_csv("training_data.csv")

# --- Identify players who actually have Stockfish evaluations ---
evaluated_players = (
    extract_df[extract_df["Eval_List"].notna() & (extract_df["Eval_List"] != "[]")]["username"]
    .unique()
)

# --- Keep only evaluated players in training data ---
filtered_training_df = training_df[training_df["username"].isin(evaluated_players)].reset_index(drop=True)

# --- Drop Endgame_ACPL column if it exists ---
if "Endgame_ACPL" in filtered_training_df.columns:
    filtered_training_df = filtered_training_df.drop(columns=["Endgame_ACPL"])

# --- Save the filtered dataset ---
filtered_training_df.to_csv("training_data_filtered.csv", index=False)

print(f"✅ Filtered training data saved with {len(filtered_training_df)} evaluated players (Endgame_ACPL removed).")



✅ Filtered training data saved with 275 evaluated players (Endgame_ACPL removed).


In [19]:
import pandas as pd
import numpy as np
import json

# --- Load datasets ---
extract_df = pd.read_csv("feature_extract.csv")
training_df = pd.read_csv("training_data_filtered.csv")

# --- Normalize usernames for matching ---
extract_df["username"] = extract_df["username"].astype(str).str.strip().str.lower()
training_df["username"] = training_df["username"].astype(str).str.strip().str.lower()

GAMES_PER_PLAYER = 16

for player in extract_df["username"].unique():
    player_games = extract_df[extract_df["username"] == player].head(GAMES_PER_PLAYER)
    game_material_means = []

    for _, row in player_games.iterrows():
        try:
            material_list = json.loads(row["Material_Diff_List"])
        except (TypeError, json.JSONDecodeError, KeyError):
            continue

        if not material_list:
            continue

        color = str(row.get("Color", "white")).lower()

        mat_values = np.array(material_list, dtype=float)
        if color == "black":
            mat_values = -mat_values

        avg_imbalance = np.mean(mat_values)
        game_material_means.append(avg_imbalance)

    if len(game_material_means) == 0:
        continue

    mean_imbalance = np.nanmean(game_material_means)
    std_imbalance = np.nanstd(game_material_means)

    # --- Check if username exists in training_df ---
    mask = training_df["username"] == player
    if mask.any():
        training_df.loc[mask, "Mean_Material_Imbalance"] = mean_imbalance
        training_df.loc[mask, "Std_Material_Imbalance"] = std_imbalance
        print(f"✅ Updated {player}: Mean={mean_imbalance:.2f}, Std={std_imbalance:.2f}")
    else:
        print(f"⚠️ Username '{player}' not found in training_data.csv — skipping")

# --- Save ---
training_df.to_csv("training_data_filtered.csv", index=False)
print("✅ Material imbalance features updated successfully!")


✅ Updated onetreehill: Mean=0.33, Std=2.43
✅ Updated acaya: Mean=-0.23, Std=2.56
✅ Updated bastianmunz: Mean=-0.70, Std=1.66
✅ Updated komarudin-garut: Mean=-0.34, Std=1.21
✅ Updated gregor_baron: Mean=-0.20, Std=0.99
✅ Updated aerdna87: Mean=0.84, Std=6.06
✅ Updated lu4ik: Mean=0.71, Std=2.85
✅ Updated phet: Mean=-0.14, Std=2.43
✅ Updated inthebighooy: Mean=-0.04, Std=1.18
✅ Updated acewestfall: Mean=-0.29, Std=1.28
✅ Updated izzyreal: Mean=-0.10, Std=2.61
✅ Updated comaturbo: Mean=0.38, Std=2.38
✅ Updated crytash: Mean=0.92, Std=2.19
✅ Updated mariomaster: Mean=0.03, Std=2.52
✅ Updated jebisesome: Mean=-0.03, Std=2.10
✅ Updated mhmdkarimi13: Mean=-0.68, Std=2.14
✅ Updated ivorybags: Mean=-0.05, Std=1.21
✅ Updated georgym89: Mean=0.18, Std=1.53
✅ Updated tjk125hqf: Mean=-0.07, Std=1.53
✅ Updated yanaaa: Mean=0.74, Std=1.76
✅ Updated girat: Mean=0.25, Std=1.44
✅ Updated cipo1951: Mean=-0.15, Std=1.59
✅ Updated rolling1: Mean=0.14, Std=2.23
✅ Updated sting47: Mean=0.23, Std=1.65
✅ Updat

In [20]:
import pandas as pd
import numpy as np

# --- PARAMETERS ---
LOSS_THRESHOLD = -300   # centipawn threshold for "losing position"
SUSTAINED_PLIES = 2     # must stay losing for this many consecutive plies

# --- READ FILE ---
df = pd.read_csv('feature_extract.csv')

# --- Parse Eval_List column ---
def parse_eval_list(s):
    """Convert '[35, 40, -20, ...]' string into a list of floats."""
    if isinstance(s, str):
        s = s.strip("[]")
        if not s:
            return []
        return [float(x) for x in s.split(',')]
    elif isinstance(s, (list, np.ndarray)):
        return list(map(float, s))
    return []

df['eval_list'] = df['Eval_List'].apply(parse_eval_list)

# --- Compute t_loss ---
def find_tloss(row):
    evals = np.array(row['eval_list'])
    if len(evals) == 0:
        return np.nan  # no evals available

    # Convert evaluations to player's perspective
    if row['Colour'].lower() == 'black':
        evals = -evals

    # Find first point of sustained losing position
    for i in range(len(evals) - SUSTAINED_PLIES + 1):
        window = evals[i:i+SUSTAINED_PLIES]
        if np.all(window <= LOSS_THRESHOLD):
            return i  # ply index where player starts losing

    return np.nan  # never in a losing position

df['t_loss'] = df.apply(find_tloss, axis=1)
df['has_loss'] = df['t_loss'].notna()

print(df[['username', 'Colour', 't_loss', 'has_loss']].head())


      username Colour  t_loss  has_loss
0  Onetreehill  White     NaN     False
1  Onetreehill  White    25.0      True
2  Onetreehill  White    15.0      True
3  Onetreehill  White    33.0      True
4  Onetreehill  White     NaN     False


In [21]:
import pandas as pd
import numpy as np
import math

# ---------------- PARAMETERS ----------------
LOSS_THRESHOLD = -300   # centipawns (player-perspective)
WIN_THRESHOLD = 300     # centipawns (player-perspective)
SUSTAINED_PLIES = 2     # require threshold to hold for this many consecutive plies

# ----------------- HELPERS -------------------
def parse_eval_list(s):
    """Parse string like '[35, 40, -20, ...]' into list of floats."""
    if isinstance(s, str):
        s = s.strip()
        if s.startswith('[') and s.endswith(']'):
            s_inner = s[1:-1].strip()
            if s_inner == '':
                return []
            # split on commas; also handle possible spaces
            parts = [p.strip() for p in s_inner.split(',')]
            vals = []
            for p in parts:
                if p == '':
                    continue
                try:
                    vals.append(float(p))
                except:
                    # if something odd appears, skip it
                    continue
            return vals
        else:
            # fallback: try split by comma
            parts = [p.strip() for p in s.split(',') if p.strip() != '']
            vals = []
            for p in parts:
                try:
                    vals.append(float(p))
                except:
                    continue
            return vals
    elif isinstance(s, (list, np.ndarray)):
        return [float(x) for x in s]
    else:
        return []

def normalize_username_col(df, colnames_try):
    """Find username column from candidates, normalize, and return column name used."""
    for cname in colnames_try:
        if cname in df.columns:
            norm = df[cname].astype(str).str.strip().str.lower()
            # replace empty strings with NaN
            norm = norm.replace('', np.nan)
            df['username_norm'] = norm
            return cname
    raise KeyError("No username column found. Expected one of: {}".format(colnames_try))

# ----------------- READ FILES -----------------
fe_path = 'feature_extract.csv'
tr_path = 'training_data_filtered.csv'

fe = pd.read_csv(fe_path)
training = pd.read_csv(tr_path)

# ----------------- NORMALIZE USERNAMES -----------------
# handle possible column name variants
username_col_fe = normalize_username_col(fe, ['username','Username','user','User','player'])
username_col_tr = normalize_username_col(training, ['username','Username','user','User','player'])

# We'll merge on 'username_norm'
# but keep original username columns intact in training

# ----------------- PARSE EVAL LIST -----------------
if 'Eval_List' not in fe.columns and 'eval_list' in fe.columns:
    fe['Eval_List'] = fe['eval_list']

assert 'Eval_List' in fe.columns, "feature_extract.csv must contain column 'Eval_List'"

fe['eval_list_parsed'] = fe['Eval_List'].apply(parse_eval_list)

# normalize Colour column name (look for common variants)
colour_col = None
for c_name in ['Colour','color','Color','side','Side']:
    if c_name in fe.columns:
        colour_col = c_name
        break
if colour_col is None:
    raise KeyError("No colour column found in feature_extract.csv (expected 'Colour' or 'color').")

# normalize result column name
result_col = None
for r_name in ['Result','result','game_result','GameResult']:
    if r_name in fe.columns:
        result_col = r_name
        break
if result_col is None:
    raise KeyError("No Result column found in feature_extract.csv (expected 'Result').")

# ----------------- COMPUTE t_loss and t_win -----------------
def find_thresholds_for_row(eval_list, color):
    """Return (t_loss_index_or_nan, t_win_index_or_nan). 0-based ply index."""
    if not isinstance(eval_list, list) or len(eval_list) == 0:
        return (np.nan, np.nan)
    evals = np.array(eval_list, dtype=float)
    # convert to player's perspective: positive = player better
    if isinstance(color, str) and color.strip().lower() == 'black':
        evals = -evals
    # find first sustained losing (<=LOSS_THRESHOLD)
    t_loss = np.nan
    t_win = np.nan
    L = len(evals)
    for i in range(L - SUSTAINED_PLIES + 1):
        window = evals[i:i+SUSTAINED_PLIES]
        if np.all(window <= LOSS_THRESHOLD) and math.isnan(t_loss):
            t_loss = i
        if np.all(window >= WIN_THRESHOLD) and math.isnan(t_win):
            t_win = i
        if (not math.isnan(t_loss)) and (not math.isnan(t_win)):
            break
    return (t_loss, t_win)

# apply rowwise
fe[['t_loss','t_win']] = fe.apply(lambda r: pd.Series(find_thresholds_for_row(r['eval_list_parsed'], r[colour_col])), axis=1)
fe['has_loss'] = fe['t_loss'].notna()
fe['has_win'] = fe['t_win'].notna()

# ----------------- PER-GAME METRICS -----------------
def compute_fighting_cap(row):
    evals = row['eval_list_parsed']
    if not isinstance(evals, list) or len(evals) == 0 or pd.isna(row['t_loss']):
        return np.nan
    total_plies = len(evals)
    # number of plies after t_loss (including the ply at t_loss? we used len - t_loss)
    plies_after = total_plies - int(row['t_loss'])
    # proportion of remaining plies (player kept playing)
    return plies_after / total_plies

# standardize player result w.r.t. player perspective
def player_result_from_row(row):
    res = str(row[result_col]).strip()
    col = str(row[colour_col]).strip().lower()
    if res == '1/2-1/2' or res == '1/2 - 1/2':
        return 'draw'
    if col == 'white':
        if res == '1-0':
            return 'win'
        elif res == '0-1':
            return 'loss'
        else:
            return 'draw'  # fallback
    else:  # black
        if res == '0-1':
            return 'win'
        elif res == '1-0':
            return 'loss'
        else:
            return 'draw'

fe['fighting_capability'] = fe.apply(compute_fighting_cap, axis=1)
fe['player_result'] = fe.apply(player_result_from_row, axis=1)

# comeback: only meaningful if has_loss true
fe['comeback_flag'] = np.where((fe['has_loss']) & (fe['player_result'] == 'win'), 1, 0)

# collapse: only meaningful if has_win true
fe['collapse_flag'] = np.where((fe['has_win']) & (fe['player_result'] == 'loss'), 1, 0)

# ----------------- AGGREGATE PER PLAYER (only over relevant games) -----------------
agg_rows = []
grouped = fe.groupby('username_norm')

for uname_norm, g in grouped:
    # fighting capability: mean across games where has_loss == True
    loss_games = g[g['has_loss']]
    n_loss = len(loss_games)
    if n_loss > 0:
        fighting_mean = loss_games['fighting_capability'].astype(float).mean()
    else:
        fighting_mean = np.nan

    # comeback index: mean of comeback_flag across games with has_loss
    if n_loss > 0:
        comeback_mean = loss_games['comeback_flag'].astype(float).mean()
    else:
        comeback_mean = np.nan

    # collapse index: mean of collapse_flag across games with has_win
    win_games = g[g['has_win']]
    n_win = len(win_games)
    if n_win > 0:
        collapse_mean = win_games['collapse_flag'].astype(float).mean()
    else:
        collapse_mean = np.nan

    agg_rows.append({
        'username_norm': uname_norm,
        'FightingCapability': fighting_mean,
        'n_loss_games': n_loss,
        'ComebackIndex': comeback_mean,
        'n_loss_games_for_comeback': n_loss,
        'CollapseIndex': collapse_mean,
        'n_win_games': n_win
    })

player_features = pd.DataFrame(agg_rows)

# ----------------- MERGE with training data -----------------
# make sure training has username_norm column and normalized
training['username_norm'] = training[username_col_tr].astype(str).str.strip().str.lower().replace('', np.nan)

# Merge on normalized username
merged = pd.merge(training, player_features, on='username_norm', how='left')

# If you prefer to drop username_norm after merge:
merged = merged.drop(columns=['username_norm'])

# ----------------- SAVE -----------------
merged.to_csv('training_data_filtered.csv', index=False)
print("✅ Updated training_data_filtered.csv saved. Here are a few merged rows preview:")
print(merged.head(12))




✅ Updated training_data_filtered.csv saved. Here are a few merged rows preview:
          username  MeanMoves   StdMoves  MeanEloDiff  WinRate  \
0          2700172    37.2500  15.813496      15.6250   31.250   
1              ab2    32.3750  13.657110     -40.9375   50.000   
2             abun    51.4375  16.997917     -86.0625   65.625   
3         adririen    25.9375  10.592253      44.8125   50.000   
4      acewestfall    21.5625   8.294325      69.0000   37.500   
5        aiglenoir    35.6875  17.445988      24.6875   56.250   
6         aladin86    42.9375  16.582998      94.6250   46.875   
7       alamsyahwk    43.3750  12.409002     -24.0000   75.000   
8           alarch    45.3125  19.509720      26.6875   34.375   
9    aleksandar974    35.5625  17.866053     -35.6875   46.875   
10  alexpavloxskiy    45.1875  18.163035     -28.3750   34.375   
11          alijon    38.7500  19.590814     -20.2500   68.750   

    WinRate_PosEloDiff  WinRate_NegEloDiff  Mean_ACPL   Std_A